In [ ]:
import pandas as pd
import re
import os
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import MinMaxScaler

In [ ]:
explanation_files = sorted(list(filter(re.compile("explanations.*.csv").search, os.listdir("results"))))
print("Found %s explanation files:" % len(explanation_files))
for i, f in enumerate(explanation_files):
    print("  %i. %s" % (i+1, f))
df = pd.read_csv(os.path.join("results", explanation_files[-1]))

## Utilities

#### Matplotlib settings

In [ ]:
# colormap for the plots and global settings

colormap = ListedColormap(["#D00000", "#FFBA08", "#3F88C5", "#032B43", "#136F63"], name="CMAPSS_5")

plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colormap.colors)
plt.rcParams['axes.axisbelow'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

colormap

#### Data transformations

In [ ]:
def get_features_and_explanations(df):
    n_observations = len(df[["unit", "RUL"]].value_counts())
    columns = sorted(df["Feature"].unique())
    X = pd.DataFrame(df.sort_values(by=["unit", "RUL", "Feature"])["Value"].values.reshape((n_observations, -1)), columns=columns)
    explanation_scores = pd.DataFrame(df.sort_values(by=["unit", "RUL", "Feature"])["Explain_Score"].values.reshape((n_observations, -1)), columns=columns)
    rul = pd.DataFrame(df.sort_values(by=["unit", "RUL", "Feature"])[["unit", "RUL"]]).drop_duplicates()["RUL"]
    return X, explanation_scores, rul

#### Scaler

In [ ]:
class CustomScaler:
    """ Custom scaler which scales all features to min and max found in the dataframe.
        The scaling may not be done to a certain quantile.
        
        We want to keep the scale relative, but the explanations should still be cetered around 0 -> min max scaling or other could shift
        the explanations and original 0 could become i.e. 0.2, which should be avoided.
        Therefore scaling is done in following way:
            1. We take all the importances from a given dataframes -> perfectly all dataframes for certain method i.e. shap
            2. We calculate the xth quantile (parameter), which will be considered as 1
            3. We divide all the values by this parameter -> in such way most relevant features will have importance of around 1
               and non-important will still be 0
        
    """

    def __init__(self, quantile=1.0):
        self.q = quantile
        self.xmax = None
        
    
    def fit(self, Xs):
        X = np.concatenate(Xs)
        X_flat_abs = np.abs(X.reshape((-1)))
        self.xmax = np.quantile(X_flat_abs, self.q)
    
    def transform(self, X):
        return X / self.xmax

    
    def inverse_transform(self, X_scaled):
        pass
    

def build_scaler(files, scaler_dict={}):
    exp_dfs = []
    for f in files:
        df = pd.read_csv(os.path.join("results", f))
        print(f, len(df))
        _f, explanation_scores, _r = get_features_and_explanations(df)
        exp_dfs.append(explanation_scores)
    scaler = CustomScaler(**scaler_dict)
    scaler.fit(exp_dfs)
    
    return scaler



In [ ]:
# global scalers 

shap_files = [f for f in explanation_files if 'shap' in f]
lime_files = [f for f in explanation_files if 'lime' in f]
ebm_files = [f for f in explanation_files if 'ebm' in f]

shap_scaler = build_scaler(shap_files, scaler_dict={"quantile": 0.95})
lime_scaler = build_scaler(lime_files, scaler_dict={"quantile": 0.95})
ebm_scaler = build_scaler(ebm_files, scaler_dict={"quantile": 0.95})


#### Test

In [ ]:
# test data
df = pd.read_csv(os.path.join("results", 'xgb_shap_explanations_0208-1641.csv'))
X, explanation_scores, rul = get_features_and_explanations(df)
explanation_scores_scaled = shap_scaler.transform(explanation_scores)

## Explanation metrics

In [ ]:
distances = []
for i in tqdm(range(len(X))):
    for j in range(len(X)):
        if j <= i:
            continue
        dist = np.linalg.norm(X.values[i] - X.values[j])
        if dist > 0:
            distances.append(dist)
        
plt.hist(distances, bins=50)
plt.show()

for q in [0.01, 0.02, 0.03, 0.04, 0.05, 0.1, 0.25]:
    p_q = np.quantile(distances, q)
    print("%ith qunatile = %.2f" % (q*100, p_q))

#### Stability F
From https://github.com/sbobek/inxai

In [ ]:
# stability -> the dist + 1 does not appear to be consistent with the paper

def stability(X, all_importances, epsilon=0.3, progress_bar=False):
    """Stability as Lipschitz coefficient.
    :param X:
    :param all_importances:
    :param epsilon:
    :return:
    """
    l_values = []

    if  not isinstance(all_importances, np.ndarray):
        all_importances = np.array(all_importances)

    for data_idx, (_, observation) in tqdm(enumerate(X.iterrows()), disable=not progress_bar, total=len(X)):
        max_val = 0
        for idx, (_, other_observation) in enumerate(X.iterrows()):
            dist = np.linalg.norm(observation - other_observation)
            
            if dist == 0:
                # to avoid division by 0 if distance is 0, the observation is omitted
                continue
            
            if dist < epsilon:
                delta_imp = np.linalg.norm(all_importances[data_idx] - all_importances[idx])
                val =  delta_imp / dist
                if val > max_val:
                    max_val = val
                    
        if max_val == 0:
            # there were no close points found so we do not include this point in the results
            continue
        
        l_values.append(max_val)
        
    return np.array(l_values)


In [ ]:
# test
explanations_stability = stability(X, explanation_scores_scaled, epsilon=3.0, progress_bar=True)
plt.hist(explanations_stability)
plt.show()

## Calculations and Plots

#### Consistency (from inxai / sbobek)

In [ ]:
def consistency(all_importances_per_model, confidence=None, progress_bar=False):
    """ Calculates maximum distance to explanation generated by the same instance and different models
    :param all_importances_per_model:
    :return:
    """
    c_values = []

    if  not isinstance(all_importances_per_model, np.ndarray):
        all_importances_per_model = np.array(all_importances_per_model)
        

    if confidence is None:
        confidence = np.ones((all_importances_per_model.shape[0],all_importances_per_model.shape[1]))
    elif not isinstance(confidence,np.ndarray):
        confidence = np.array(confidence).reshape((all_importances_per_model.shape[0],all_importances_per_model.shape[1]))

    for obs_idx in tqdm(range(len(all_importances_per_model[0])), disable=not progress_bar):
        largest_dist = 0
        for model_idx, model_imps in enumerate(all_importances_per_model):
            for other_idx, compared_model in enumerate(all_importances_per_model[:model_idx]):
                current_imps = model_imps[obs_idx]
                other_imps = compared_model[obs_idx]
                if not isinstance(current_imps, np.ndarray):
                    current_imps = np.array(current_imps)
                if not isinstance(other_imps, np.ndarray):
                    other_imps = np.array(other_imps)
                dist = np.linalg.norm(current_imps - other_imps)*confidence[model_idx,obs_idx]*confidence[other_idx,obs_idx]
                if dist > largest_dist:
                    largest_dist = dist
        c_values.append(1.0/(largest_dist+1))

    return c_values


In [ ]:
# test
shap_consistency = consistency([explanation_scores_scaled, explanation_scores_scaled])

### Stability

In [ ]:
try:
    with open("results/stability_dict.pickle", "rb") as f:
        stability_dict = pickle.load(f)
except Exception as e:
    print(e)
    stability_dict = {}

In [ ]:
# 'data' dictionary is structured in following way:
# key -> this is the name of the explanation model i.e. xgb_lime
# value -> this is tuple containing (df, es), where df contains feature values and es contains explanation scores (feature importances)

data = {}

for f in tqdm(explanation_files):
    df = pd.read_csv(os.path.join("results", f))
    X, es, rul = get_features_and_explanations(df)
    
    if "shap" in f:
        # print("Scaling with shap...")
        es_scaled = shap_scaler.transform(es)
    elif "lime" in f:
        # print("Scaling with lime...")
        es_scaled = lime_scaler.transform(es)
    elif "ebm" in f:
        # print("Scaling with ebm...")
        es_scaled = ebm_scaler.transform(es)
    else:
        raise ValueError("One of explanation methods ('shap', 'lime', 'ebm') must be in file name to assign correct scaler.")
    
    # name = f.replace("_explanations.csv", "")
    name = re.sub("_explanations.*.csv", "", f)
    data[name] = (X, es_scaled, rul)
    
data.keys()

In [ ]:
# parallel computation
from joblib import Parallel, delayed, cpu_count
import warnings

def calc_stability(data, key):
    print("Started calculation of stability for %s" % key)
    X, es, rul = data[key]
    stab = stability(X, es, epsilon=1.5, progress_bar=True)
    print("Stability for %s calculated" % key)
    return {key: stab}

results = Parallel(n_jobs=6)(delayed(calc_stability)(data, i) for i in data.keys())

stability_dict = {}
for element in results:
    key = list(element.keys())[0]
    stability_dict[key] = element[key]

with open(r"results/stability_dict_ebm.pickle", "wb") as f:
    pickle.dump(stability_dict, f)

In [ ]:
def stability_dict_to_df(stability_dict):
    """Transforms stability_dict to dataframe for summary boxplot"""
    df_stability = pd.DataFrame.from_dict(stability_dict).melt(var_name="name", value_name="stability")
    df_stability["classification_method"] = df_stability["name"].apply(lambda x: x.split("_")[0].upper())    
    df_stability["explanation_method"] = df_stability["name"].apply(lambda x: "SHAP" if "shap" in x else
                                                                     "LIME" if "lime" in x else
                                                                     "EBM" if "ebm" in x else "error")
    
    return df_stability

    
df_stability = stability_dict_to_df(stability_dict)

plt.figure(figsize=(8, 3.5))
plt.rc('axes', axisbelow=True)
#plt.gca().yaxis.grid(alpha=1, zorder=1)
sns.boxplot(data=df_stability, x="classification_method", y="stability", hue="explanation_method", hue_order=["LIME", "EBM", "SHAP"], width=0.5,
           fliersize=2, linewidth=1)
plt.xlabel(None)
plt.legend(bbox_to_anchor=(0.5, 1.05), ncol=3, loc="center", frameon=False, title=None, title_fontsize=11)
plt.gca().yaxis.grid(alpha=0.8)
#plt.yscale('log')
plt.ylim(0, )
plt.tight_layout()
# plt.savefig(r"plots/models_stability.png", dpi=300)

#### Stability vs epsilon

In [ ]:
#stability_dict_eps = {}

epsilons = [0.005, 0.01]
# epsilons = [0.05, 0.1]
# epsilons = [0.2, 0.3, 0.5]
# epsilons = [0.8, 1]


for eps in tqdm(epsilons):
    for key, result in data.items():
        X, es = result
        key_eps = key + "_" + str(eps)
        stability_dict_eps[key_eps] = stability(X, es, epsilon=eps, progress_bar=False)
        
with open(r"results/stability_dict_eps.pickle", "wb") as f:
    pickle.dump(stability_dict_eps, f)

In [ ]:
df_stability_eps = pd.DataFrame()

i = 0
for key, stability_score in stability_dict_eps.items():
    mod = re.findall("xgb|rf|mlp|svm|ebm", key)[0]
    explain = re.findall("shap|lime|ebm", key)[0]
    eps = key.split("_")[-1]
    
    df_stability_eps.loc[i, "Model"] = mod.upper()
    df_stability_eps.loc[i, "Explanation"] = explain.upper()
    df_stability_eps.loc[i, "Model+Explanation"] = "%s+%s" % (mod.upper(), explain.upper())
    df_stability_eps.loc[i, "Epsilon"] = float(eps)
    df_stability_eps.loc[i, "Stability"] = np.mean(stability_score)
    
    i += 1
    
sns.lineplot(data=df_stability_eps.query("Explanation == 'SHAP'"), x="Epsilon", y="Stability", hue="Model")

#### Consistency

In [ ]:
consistency_dict = {}

for m1, res1 in tqdm(data.items()):
    X1, es1, rul1 = res1
    for m2, res2 in data.items():
        X2, es2, rul2 = res2
        # we do not compare the model to itself, as the consistency here would be always 1
        if m1 == m2:
            continue
        
        key = (m1, m2)
        key_inverse = (m2, m1)
        
        # if a pair was already calculated, skip it
        if key_inverse in consistency_dict.keys():
            continue
        
        consistency_dict[key] =  consistency([es1, es2])


In [ ]:
def consistency_dict_to_df(consistency_dict, aggregate_method=np.mean):
    names = sorted(set(map(lambda x: x[0], consistency_dict.keys())))
    names = [n.replace("_", "+").upper() for n in names]
    df_consistency = pd.DataFrame(index=names, columns=names)
    
    for name_tup, result in consistency_dict.items():
        m1, m2 = name_tup
        m1 = m1.replace("_", "+").upper()
        m2 = m2.replace("_", "+").upper()
        df_consistency.loc[m1, m2] = aggregate_method(result)
    
    df_consistency = df_consistency.astype(np.float32)
    df_consistency = df_consistency.iloc[:, 1:] # removes first column, which has only NaNs
    return df_consistency

df_consistency = consistency_dict_to_df(consistency_dict, aggregate_method=np.median)

plt.figure(figsize=(6, 6))
sns.heatmap(df_consistency, cmap="viridis", annot=True ,
            cbar_kws={"label": "Consistency", "location": "top", "fraction": 0.06, "format": "%.2f"},
            vmin=0.2, vmax=0.6)
plt.tight_layout()
plt.savefig(r"plots/models_consistency.png", dpi=300)

### Other plots

#### Most important features

In [ ]:
important_features = []
for key in data.keys():
    es = data[key][1]
    top_features = es.abs().mean(axis=0).sort_values(ascending=False)
    top_features = top_features.index.values[:3]
    for feat in top_features:
        if feat not in important_features:
            important_features.append(feat)

df_feature_imp = pd.DataFrame()

i = 0
for key in data.keys():
    es = data[key][1]
    for feat in sorted(important_features):
        df_feature_imp.loc[i, "Explanation"] = key.replace("_", "+").upper()
        df_feature_imp.loc[i, "Feature"] = feat
        df_feature_imp.loc[i, "Importance"] = es[feat].abs().mean()
        i+=1

In [ ]:
#sns.set_palette(colormap.colors, n_colors=5)
plt.figure(figsize=(10, 3))
sns.barplot(hue="Feature", y="Importance", x="Explanation", data=df_feature_imp)
plt.legend(bbox_to_anchor=(0.5, 1.05), ncol=5, loc="center", frameon=False, title=None, title_fontsize=11)
plt.xlabel("")
plt.ylabel("Mean relative importance")
#plt.gca().yaxis.grid(alpha=0.4)
plt.tight_layout()
plt.savefig(r"plots/important_features.png", dpi=300)

#### Change of feature importance with RUL

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.ticker import FormatStrFormatter

# get all availabe units and features
units = df["unit"].unique()
features = df["Feature"].unique()

# choose unit and feature
u = np.random.choice(units)
#u = 84
feature = "Measurement 14"


# global line properties for each plot
marker = 'o'
markeredgecolor="white"
linewidth = 1
model_colors = {"xgb": colormap.colors[0], "rf": colormap.colors[1], "mlp": colormap.colors[2], "svm": colormap.colors[3], "ebm": colormap.colors[4]}

plt.figure(figsize=(8, 7))
# plot the measurement on the first subplot
df = pd.read_csv(os.path.join("results", explanation_files[0]))
dfu = df.loc[(df["unit"] == u)  & (df["Feature"] == feature)]

plt.subplot(411)
plt.plot(dfu["RUL"], dfu["Value"], color="black", marker=marker, markersize=4, markeredgecolor=markeredgecolor, linewidth=0.5)
plt.xlim(dfu["RUL"].max(), 0)
plt.ylabel("Feature value")
plt.title("%s - unit %s" % (feature, u), loc='left')
# plt.xlabel("RUL (cycles)")

# build a global legend for models:
legend_elements = [Line2D([0], [0], linewidth=linewidth, 
                          marker=marker, 
                          markeredgecolor=markeredgecolor, 
                          linestyle='-', 
                          color=col,
                          label=key.upper()) for key, col in model_colors.items()]

plt.legend(handles=legend_elements, bbox_to_anchor=(0.5, 1.4), ncol=5, loc="center", frameon=False)


# make some global plots for each explanation
for i, exp in enumerate(["SHAP", "LIME", "EBM"]):
    plt.subplot(412+i)
    plt.title("%s" % exp, loc='left')
    # plt.plot(dfu["RUL"], np.zeros(dfu["RUL"].shape[0]), color="black")
    plt.xlim(dfu["RUL"].max(), 0)
    plt.ylabel("Explanation score")
    
    plt.gca().yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

# xlabel only on last plot
plt.xlabel("RUL (cycles)")

for f in explanation_files:
    # read each explanation file and extract selected unit / feature
    df = pd.read_csv(os.path.join("results", f))
    dfu = df.loc[(df["unit"] == u)  & (df["Feature"] == feature)]
      
    # based on file name get model and explanation algorithm
    mod = re.findall("xgb|rf|mlp|svm|ebm", f)[0]
    explain = re.findall("shap|lime|ebm", f)[0]
    if explain == "shap":
        plt.subplot(412) 
    elif explain == 'lime':
        plt.subplot(413)
    elif explain == "ebm":
        plt.subplot(414)
    color = model_colors[mod]
    plt.plot(dfu["RUL"], dfu["Explain_Score"], marker='o', color=color, linewidth=1., linestyle='-', markeredgecolor="white", markersize=4,)
    
    
plt.tight_layout()
#plt.savefig(r"plots/unit_explanations.png", dpi=300)

## Dimensionality Reduction

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

tsne = TSNE(**{"n_components": 2, "perplexity": 50})
pca = PCA(n_components=2)


def plot_2d(data, keys, titles, dim_model, save=False):
    if type(keys) != list:
        keys = [keys]
    if type(titles) != list:
        titles = [titles]
        
    fig, axes = plt.subplots(nrows=1, ncols=1+len(keys), figsize=(10, 4))
    data_plotted = False
    
    for i, (key, title) in enumerate(zip(keys, titles)):
        data_explain = data[key]
        X_2d_data = dim_model.fit_transform(data_explain[0])
        X_2d_exp = dim_model.fit_transform(data_explain[1])
    
        if not data_plotted:
            axes[0].set_title("Measurements")
            axes[0].scatter(X_2d_data[:, 0], X_2d_data[:, 1], s=8, c=data_explain[2], cmap='viridis', vmin=0, vmax=130)
            data_plotted = True
    
        axes[i+1].set_title(title)
        scatter = axes[i+1].scatter(X_2d_exp[:, 0], X_2d_exp[:, 1], s=8, c=data_explain[2], cmap='viridis', vmin=0, vmax=130)
    
#     for ax in axes.flat:
#         ax.set_xticks([])
#         ax.set_yticks([])
    for ax in axes.flat:
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
    
    fig.colorbar(scatter, ax=axes.ravel().tolist(), label="RUL", location="bottom", pad=0.15, shrink=0.5, fraction=0.1, aspect=30)
    #fig.tight_layout(rect=[0.1, 0, 0.8, 0.9])
    if save:
        dim_name = dim_model.__repr__().split("(")[0].lower()
        key_names = "+".join(keys)
        plt.savefig(rf"plots/{dim_name}-{key_names}.png", dpi=300)
    plt.show()
    
    
plot_2d(data, ["mlp_shap", "mlp_lime"], ["SHAP", "LIME"], pca, save=True)

In [ ]:
for key in data.keys():
    print(key)
    plot_2d(data, key, key, pca)